# AFL Model - Part 1 - Data Cleaning

These tutorials will walk you through how to construct your own basic AFL model, using publicly available data. The output will be odds for each team to win, which will be shown on [The Hub](https://www.betfair.com.au/hub/tools/models/afl-prediction-model/).

In this notebook we will walk you through the basics of cleaning this dataset and how we have done it. If you want to get straight to feature creation or modelling, feel free to jump ahead!

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import re

We will first explore the DataFrames, and then create functions to wrangle them and clean them into more consistent sets of data.

In [2]:
# Read/clean each DataFrame
match_results = pd.read_csv("data/afl_match_results.csv")
odds = pd.read_csv("data/afl_odds.csv")

# Read the historical player stats DataFrames together and append them
player_stats = pd.read_csv("data/player_stats_2010.csv")
player_stats_2018 = pd.read_csv("data/player_stats_2018.csv")
col_order = player_stats.columns
player_stats = player_stats.append(player_stats_2018)[col_order]

Have a look at the structure of the DataFrames. Notice that for the odds DataFrame, each game is split between two rows, whilst for the match_results each game is on one row. We will have to get around this by splitting the games up onto two rows, as this will allow our feature transformation functions to be applied more easily later on. For the player_stats DataFrame we will aggregate these stats into each game on separate rows.

In [3]:
match_results.head(3)

,Game,Date,Round,Home.Team,Home.Goals,Home.Behinds,Home.Points,Away.Team,Away.Goals,Away.Behinds,Away.Points,Venue,Margin,Season,Round.Type,Round.Number
0,1,1897-05-08,R1,Fitzroy,6,13,49,Carlton,2,4,16,Brunswick St,33,1897,Regular,1
1,2,1897-05-08,R1,Collingwood,5,11,41,St Kilda,2,4,16,Victoria Park,25,1897,Regular,1
2,3,1897-05-08,R1,Geelong,3,6,24,Essendon,7,5,47,Corio Oval,-23,1897,Regular,1


In [4]:
odds.tail(3)

,trunc,event_name,path,selection_name,odds
4063,2018-07-15,Match Odds,AFL/Fremantle v Port Adelaide,Port Adelaide Power,1.2047
4064,2018-07-15,Match Odds,AFL/North Melbourne v Sydney,North Melbourne,1.9263
4065,2018-07-15,Match Odds,AFL/North Melbourne v Sydney,Sydney,2.0647


In [5]:
player_stats.tail(3)

,Date,Season,Round,Venue,Player,Team,Opposition,Status,GA,Match_id,...,FA,AF,SC,CCL,SCL,SI,MG,TO,ITC,T5
6333,15/07/2018,2018,Round 17,Optus Stadium,Dougal Howard,Port Adelaide,Fremantle,Away,0,9657,...,2,36,58,0.0,0.0,1.0,229.0,5.0,7.0,0.0
6334,15/07/2018,2018,Round 17,Optus Stadium,Robbie Gray,Port Adelaide,Fremantle,Away,0,9657,...,0,25,37,0.0,2.0,1.0,128.0,1.0,0.0,0.0
6335,15/07/2018,2018,Round 17,Optus Stadium,Patrick Ryder,Port Adelaide,Fremantle,Away,0,9657,...,0,51,49,0.0,0.0,1.0,-30.0,0.0,1.0,0.0


First, we will write functions to make the odds data look a bit nicer, with home and away team columns and a date column. To do this we will use the [regex](https://docs.python.org/3/howto/regex.html) module to extract the team names from the path column, as well as the to_datetime function from pandas. We will also replace all the inconsistent team names with consistent team names

In [6]:
def odds_wrangling(df):
    # Create a date column
    df['Date'] = pd.to_datetime(df['trunc']).dt.date
    
    # Grab the home and away teams using regex from the match_results column
    df['home_team'] = df['path'].str.extract('(([\w\s]+) v ([\w\s]+))', expand=True)[1].str.strip()
    df['away_team'] = df['path'].str.extract('(([\w\s]+) v ([\w\s]+))', expand=True)[2].str.strip()
    df['match_details'] = df['path'].str.extract('(([\w\s]+) v ([\w\s]+))', expand=True)[0].str.strip()
    
    # Drop unneeded columns
    df = df.drop(columns=['path', 'trunc', 'event_name', 'match_details'])
    
    # Rename column
    df = df.rename(columns={'selection_name': 'Team'})
    return df

def clean_odds(df):
    # Clean team names to be consistent across DataFrames
    df = df.replace(
    {
        'Adelaide Crows': 'Adelaide',
        'Brisbane Lions': 'Brisbane',
        'Carlton Blues': 'Carlton',
        'Collingwood Magpies': 'Collingwood',
        'Essendon Bombers': 'Essendon',
        'Fremantle Dockers': 'Fremantle',
        'GWS Giants': 'GWS',
        'Geelong Cats': 'Geelong',
        'Gold Coast Suns': 'Gold Coast',
        'Greater Western Sydney': 'GWS',
        'Greater Western Sydney Giants': 'GWS',
        'Hawthorn Hawks': 'Hawthorn',
        'Melbourne Demons': 'Melbourne', 
        'North Melbourne Kangaroos': 'North Melbourne',
        'Port Adelaide Magpies': 'Port Adelaide',
        'Port Adelaide Power': 'Port Adelaide', 
        'P Adelaide': 'Port Adelaide',
        'Richmond Tigers': 'Richmond',
        'St Kilda Saints': 'St Kilda', 
        'Sydney Swans': 'Sydney',
        'West Coast Eagles': 'West Coast',
        'Wetsern Bulldogs': 'Western Bulldogs',
        'Western Bullbogs': 'Western Bulldogs'
    }
    )
    return df

In [7]:
# Apply the wrangling and cleaning
odds = odds_wrangling(odds)
odds = clean_odds(odds)
odds.tail()

,Team,odds,Date,home_team,away_team
4061,West Coast,2.7815,2018-07-15,Collingwood,West Coast
4062,Fremantle,5.9110,2018-07-15,Fremantle,Port Adelaide
4063,Port Adelaide,1.2047,2018-07-15,Fremantle,Port Adelaide
4064,North Melbourne,1.9263,2018-07-15,North Melbourne,Sydney
4065,Sydney,2.0647,2018-07-15,North Melbourne,Sydney


We now have a DataFrame that looks nice and easy to join with our other DataFrames. Let's clean the names so the join is easier, then we'll fix up the match_details DataFrame.

In [8]:
def match_results_wrangling(df):
    # Create DataFrame which includes all the home teams' statistics, as well as the stats for the away team (Opposition)
    df_home = pd.DataFrame(
        {
            'Game': df['Game'],
            'Date': df['Date'],
            'Round': df['Round.Number'],
            'Team': df['Home.Team'],
            'Goals': df['Home.Goals'],
            'Behinds': df['Home.Behinds'],
            'Points': df['Home.Points'],
            'Margin': df['Margin'],
            'Venue': df['Venue'],
            'Home?': 1,
            'Opposition': df['Away.Team'],
            'Opposition Goals': df['Away.Goals'],
            'Opposition Behinds': df['Away.Behinds'],
            'Opposition Points': df['Away.Points']
    })
    # Create DataFrame which includes all the away teams' statistics, as well as the stats for the home team (Opposition)
    df_away = pd.DataFrame(
        {
            'Game': df['Game'],
            'Date': df['Date'],
            'Round': df['Round.Number'],
            'Team': df['Away.Team'],
            'Goals': df['Away.Goals'],
            'Behinds': df['Away.Behinds'],
            'Points': df['Away.Points'],
            'Margin': - df['Margin'],
            'Venue': df['Venue'],
            'Home?': 0,
            'Opposition': df['Home.Team'],
            'Opposition Goals': df['Home.Goals'],
            'Opposition Behinds': df['Home.Behinds'],
            'Opposition Points': df['Home.Points']
    })
    
    # Append the DataFrames together, then sort by the Game ID so that we have the same game on consecutive rows
    df = df_home.append(df_away).sort_values(by='Game').reset_index(drop=True)
    
    # Change the Date column to a Datetime object
    df['Date'] = pd.to_datetime(df['Date']).dt.date
    return df

# Define a function which cleans the match_results DataFrame
def clean_match_results(df):
    # Clean team names to be consistent across DataFrames
    df = df.replace(
    {
        'Brisbane Lions': 'Brisbane',
        'Footscray': 'Western Bulldogs'
    }
    )  
    return df

In [9]:
match_results = match_results_wrangling(match_results)
match_results = clean_match_results(match_results)
match_results.head()

,Behinds,Date,Game,Goals,Home?,Margin,Opposition,Opposition Behinds,Opposition Goals,Opposition Points,Points,Round,Team,Venue
0,13,1897-05-08,1,6,1,33,Carlton,4,2,16,49,1,Fitzroy,Brunswick St
1,4,1897-05-08,1,2,0,-33,Fitzroy,13,6,49,16,1,Carlton,Brunswick St
2,11,1897-05-08,2,5,1,25,St Kilda,4,2,16,41,1,Collingwood,Victoria Park
3,4,1897-05-08,2,2,0,-25,Collingwood,11,5,41,16,1,St Kilda,Victoria Park
4,6,1897-05-08,3,3,1,-23,Essendon,5,7,47,24,1,Geelong,Corio Oval


Now we have both the odds DataFrame and match_results DataFrame ready for feature creation! Finally, we will aggregate the player_stats DataFrame stats for each game rather than individual player stats. For this DataFrame we have regular stats, such as disposals, marks etc. and Advanced Stats, such as Tackles Inside 50 and Metres Gained. However these advanced stats are only available from 2015, so we will not be using them in this tutorial - as there isn't enough data from 2015 to train our models.

Let's now aggregate the player_stats DataFrame.

In [10]:
def player_stats_wrangling(df):
    # Aggregate the stats
    agg_stats = df.groupby(by=['Date', 'Season', 'Round', 'Team', 'Opposition', 'Status'], as_index=False).sum()

    # Drop irrelevant columns such as Disposal Efficiency and Time On Ground which are meaningless when aggregated
    agg_stats = agg_stats.drop(columns=['DE', 'TOG', 'Match_id'])
    
    # Change the Date column to a Datetime object
    agg_stats['Date'] = pd.to_datetime(agg_stats['Date']).dt.date
    return agg_stats

In [11]:
agg_stats = player_stats_wrangling(player_stats)

We now have a three fully prepared DataFrames which are almost ready to be analysed and for a model to be built on! Let's have a look at how they look and then merge them together into our final DataFrame.

In [12]:
odds.tail(3)

,Team,odds,Date,home_team,away_team
4063,Port Adelaide,1.2047,2018-07-15,Fremantle,Port Adelaide
4064,North Melbourne,1.9263,2018-07-15,North Melbourne,Sydney
4065,Sydney,2.0647,2018-07-15,North Melbourne,Sydney


In [13]:
match_results.tail(3)

,Behinds,Date,Game,Goals,Home?,Margin,Opposition,Opposition Behinds,Opposition Goals,Opposition Points,Points,Round,Team,Venue
30685,8,2018-07-15,15343,15,1,-6,Sydney,8,16,104,98,17,North Melbourne,Docklands
30686,11,2018-07-15,15344,8,1,9,Port Adelaide,8,7,50,59,17,Fremantle,Perth Stadium
30687,8,2018-07-15,15344,7,0,-9,Fremantle,11,8,59,50,17,Port Adelaide,Perth Stadium


In [14]:
agg_stats.tail(3)

,Date,Season,Round,Team,Opposition,Status,GA,CP,UP,ED,...,FA,AF,SC,CCL,SCL,SI,MG,TO,ITC,T5
3569,2018-09-06,2018,Round 12,North Melbourne,Geelong,Away,5,140,202,253,...,18,1350,1455,14.0,22.0,74.0,5157.0,76.0,66.0,9.0
3570,2018-09-06,2018,Round 12,St Kilda,Sydney,Home,5,124,220,244,...,22,1463,1435,12.0,15.0,92.0,5024.0,50.0,52.0,6.0
3571,2018-09-06,2018,Round 12,Sydney,St Kilda,Away,16,125,275,311,...,20,1682,1866,14.0,25.0,153.0,5790.0,52.0,50.0,2.0


In [15]:
# Create a function which merges the DataFrames
def merge_dfs(odds_df, match_results_df, agg_stats_df):
    # Before we merge the DataFrames, let's filter out games that aren't played between teams in our agg_stats_df
    teams = agg_stats_df['Team'].unique()
    odds_df = odds_df[(odds_df['home_team'].isin(teams)) & (odds_df['away_team'].isin(teams))]
    
    # Merge the odds DataFrame with match_results
    df = pd.merge(odds_df, match_results_df, how='inner', on=['Team', 'Date'])
    
    # Merge that df with agg_stats
    df = pd.merge(df, agg_stats_df, how='inner', on=['Team', 'Date'])
    
    # Sort the values so that each game is ordered by Date
    df = df.sort_values(by=['Game', 'Home?']).reset_index(drop=True)
    
    # Drop duplicate columns and rename these
    df = df.drop(columns=['Round_y', 'Opposition_y']).rename(columns={'Opposition_x': 'Opposition', 'Round_x': 'Round'})
    return df

In [16]:
afl_data = merge_dfs(odds, match_results, agg_stats)

In [17]:
afl_data.tail(3)

,Team,odds,Date,home_team,away_team,Behinds,Game,Goals,Home?,Margin,...,FA,AF,SC,CCL,SCL,SI,MG,TO,ITC,T5
3095,North Melbourne,1.9263,2018-07-15,North Melbourne,Sydney,8,15343,15,1,-6,...,9,1557,1643,16.0,20.0,98.0,5700.0,62.0,57.0,6.0
3096,Port Adelaide,1.2047,2018-07-15,Fremantle,Port Adelaide,8,15344,7,0,-9,...,23,1495,1573,9.0,30.0,57.0,5664.0,85.0,76.0,9.0
3097,Fremantle,5.9110,2018-07-15,Fremantle,Port Adelaide,11,15344,8,1,9,...,14,1572,1726,9.0,25.0,76.0,6018.0,76.0,85.0,16.0


Great! We now have a clean looking datset with each row representing one team in a game. Let's now eliminate the outliers from a dataset. We know that Essendon had a doping scandal which resulted in their entire team being banned for a year in 2016, so let's remove all of their 2016 games. To do this we will filter based on the team and season, and then invert this with ~.

In [18]:
# Define a function which eliminates outliers
def outlier_eliminator(df):
    # Eliminate Essendon 2016 games
    essendon_filter_criteria = ~(((df['Team'] == 'Essendon') & (df['Season'] == 2016)) | ((df['Opposition'] == 'Essendon') & (df['Season'] == 2016)))
    df = df[essendon_filter_criteria]
    
    # Reset index
    df = df.reset_index(drop=True)
    return df

In [19]:
afl_data = outlier_eliminator(afl_data)

Our data is now fully ready to be explored and for features to be created, which we will walk you through in our next tutorial, [AFL Feature Creation Tutorial](02. afl_feature_creation_tutorial.ipynb).